In [80]:

!pip3 install --upgrade --quiet langchain langchain-community langchain-ollama chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv

You should consider upgrading via the 'C:\Users\sonik\Desktop\summarizer\myenv\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\sonik\Desktop\summarizer\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [81]:
# LangChain modules
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv


In [82]:
load_dotenv()

True

In [83]:

OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

DEFINE THE LLM


In [84]:
from langchain_community.llms import Ollama

# Initialize the model
llm = Ollama(model="mistral")

# Example
response = llm.invoke("Summarize: Artificial intelligence is transforming industries.")
print(response)


 Artificial Intelligence (AI) is significantly reshaping various sectors by automating tasks, improving efficiency, and driving innovation. It's making a substantial impact in areas such as healthcare, finance, transportation, and manufacturing, among others. AI is also enabling advancements in machine learning, natural language processing, and robotics, which are expected to further revolutionize industries in the future.


Process PDF document


In [85]:
# loading the document 
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

Ignoring wrong pointing object 18 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'creator': 'Preview', 'creationdate': "D:20240909152042Z00'00'", 'author': 'Thu Vu', 'moddate': "D:20240910141854Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligen

Split the document

In [86]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "]) #page brake, line brake, space
chunks = text_splitter.split_documents(pages)

Creating Embeddings


In [87]:
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings
embedding_function=get_embedding_function()
test_vector=embedding_function.embed_query("cat")

In [88]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Ambsterdam", reference="coffeeshop")

{'score': 0.6537334985684395}

#### Create Vector Databases

In [93]:
# import uuid

# def create_vectorstore(chunks, embedding_function, vectorstore_path):

#     # Create a list of unique ids for each document based on the content
#     ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
#     # Ensure that only unique docs with unique ids are kept
#     unique_ids = set()
#     unique_chunks = []
    
#     unique_chunks = [] 
#     for chunk, id in zip(chunks, ids):     
#         if id not in unique_ids:       
#             unique_ids.add(id)
#             unique_chunks.append(chunk) 

#     # Create a new Chroma database from the documents
#     vectorstore = Chroma.from_documents(documents=unique_chunks, 
#                                         ids=list(unique_ids),
#                                         embedding=embedding_function, 
#                                         persist_directory = vectorstore_path)

#     vectorstore.persist()
    
#     return vectorstore
import uuid
from langchain_community.vectorstores import Chroma

def create_vectorstore(chunks, embedding_function, vectorstore_path):
    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure only unique docs are kept
    unique_ids = set()
    unique_chunks = []
    final_ids = []

    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk)
            final_ids.append(id)

    # Create Chroma database
    vectorstore = Chroma.from_documents(
        documents=unique_chunks,
        ids=final_ids,
        embedding=embedding_function, 
        persist_directory=vectorstore_path
    )

    vectorstore.persist()
    return vectorstore


# ✅ Save and load from the same directory
vectorstore_path = "vectorstore_test"

# Create and persist
vectorstore = create_vectorstore(
    chunks=chunks, 
    embedding_function=embedding_function, 
    vectorstore_path=vectorstore_path
)

# Load again
vectorstore = Chroma(
    persist_directory=vectorstore_path, 
    embedding_function=embedding_function
)

# Create retriever and query
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 5})
relevant_chunks = retriever.invoke("What is the title of the paper?")
print(relevant_chunks)



[Document(metadata={'moddate': "D:20240910141854Z00'00'", 'author': 'Thu Vu', 'total_pages': 3, 'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'page_label': '3', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'page': 2, 'creationdate': "D:20240909152042Z00'00'", 'creator': 'Preview'}, page_content='thesaurus, or for which there was no entry with the same linguistic sense were not\nreplaced. If two entries were of the same length, the replacement was chosen alphabe-\ntically. When necessary, minor modiﬁcations were made to the essay to maintain the\ngrammatical structure of a sentence (e.g. replacing ‘an’ with ‘a’ for replacement words\nbeginning with consonants). A ‘moderately complex’ version of each excerpt was created\nusing the same algorithm as above, except replacing only every third applicable word.\nExamples of the stimuli can be found in the appendix.\nEach part

### Prompt Template

In [109]:
# PROMPT_TEMPLATE = """
# You are given a set of document excerpts.

# Context:
# {context}

# Question:
# {question}

# Instructions:
# - Answer ONLY from the provided context.
# - If the answer is explicitly present (like a title), return it exactly as written.
# - Do not infer, rephrase, or guess.
# - If the context does not contain the answer, respond with: "Not found in context."
# """


PROMPT_TEMPLATE = """
You are given a set of document excerpts.

Context:
{context}

Question:
{question}

Instructions:
- Search carefully in the provided context for the exact answer.
- If the answer is explicitly stated, copy it **exactly as written** (do not paraphrase).
- If the answer is partially or indirectly mentioned, extract only the relevant part without adding anything extra.
- If the answer is completely missing, respond ONLY with: "Not found in context."

Also provide:
1. **Answer**: (the extracted text or "Not found in context")
2. **Reasoning**: Explain briefly why you chose that answer (e.g., exact match found, clearly labeled in context, or not present).
"""


In [110]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are given a set of document excerpts.

Context:
thesaurus, or for which there was no entry with the same linguistic sense were not
replaced. If two entries were of the same length, the replacement was chosen alphabe-
tically. When necessary, minor modiﬁcations were made to the essay to maintain the
grammatical structure of a sentence (e.g. replacing ‘an’ with ‘a’ for replacement words
beginning with consonants). A ‘moderately complex’ version of each excerpt was created
using the same algorithm as above, except replacing only every third applicable word.
Examples of the stimuli can be found in the appendix.
Each participant received only one excerpt. Participants were informed that the excerpt
came from a personal statement for graduate study in the Stanford English department.
They were instructed to read the passage, decide whether or not to accept the applicant,
and rate their conﬁdence in their decision on a 7-point scale. 1 They were then asked how
difﬁcult the passage

In [111]:
response = llm.invoke(prompt)
print(response)

 Answer: Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly

Reasoning: The title is explicitly stated at the beginning of the text.


Using Langchain Expression Language

In [112]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

' Answer: Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly\n\nReasoning: The title is explicitly stated at the beginning of the provided text.'

Generating Structured Responses

In [113]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class AnswerWithSources(BaseModel): 
    answer: str
    sources: str
    reasoning: str

class ExtractedInfo(BaseModel):
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

parser = JsonOutputParser(pydantic_object=ExtractedInfo)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that extracts research article details."),
    ("user", "Given the context:\n{context}\n\nExtract the following strictly in JSON: {format_instructions}\n\nQuestion: {question}")
]).partial(format_instructions=parser.get_format_instructions())

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

result = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
print(result)


{'paper_title': {'answer': 'Empirical Investigation of the Strategy of Complexity', 'sources': '[Oppenheimer, D. M., 2006]'}, 'paper_summary': {'answer': 'The paper investigates the strategy of using complex language to appear more intelligent in writing. The study finds that such a strategy is unsuccessful as it negatively impacts raters’ assessments of the text’s authors.', 'sources': '[Oppenheimer, D. M., 2006]'}, 'publication_year': {'answer': '2006', 'sources': '[Oppenheimer, D. M., 2006]'}, 'paper_authors': {'answer': 'D. M. Oppenheimer', 'sources': '[Oppenheimer, D. M., 2006]'}}


In [114]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

df = pd.DataFrame([structured_response])

answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0].get('answer', ''))
    source_row.append(df[col][0].get('sources', ''))
    reasoning_row.append(df[col][0].get('reasoning', ''))

structured_response_df = pd.DataFrame(
    [answer_row, source_row, reasoning_row],
    columns=df.columns,
    index=['answer', 'source', 'reasoning']
)

structured_response_df


,paper_title,paper_summary,publication_year,paper_authors
answer,The Strategy of Complexity in Writing,This paper investigates the strategy of using ...,2006,D.M. Oppenheimer
source,"[Oppenheimer, D. M., 2006]","[Oppenheimer, D. M., 2006]","[Oppenheimer, D. M., 2006]","[Oppenheimer, D. M., 2006]"
reasoning,,,,
